In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Getting helper function

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-23 09:01:56--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-23 09:01:56 (43.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-06-23 09:02:07--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 173.194.202.128, 74.125.199.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2023-06-23 09:02:07 (93.2 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



### Visualizing dataset

In [5]:
import pandas as pd

In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
print(f"Total training samples: {len(train_df)}")
print(f"Total testing samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total testing samples: 3263
Total samples: 10876


In [11]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [12]:
import random

In [13]:
random_index = random.randint(0, len(train_df)-5)

for index in range(random_index, random_index+5):
    print(f'Target: {train_df["target"][index]}')
    print(f'Text:\n{train_df["text"][index]}')
    print("\n")

Target: 0
Text:
Conditions for Paris FR at 4:00 am CEST: Current Conditions:
Fair 68 FForecast:
Thu - Sunny. High: 87 Low: 61
Fri - PM Thunderstorm...


Target: 1
Text:
Severe Weather Statement issued August 05 at 9:33PM CDT by NWS: ...THE SEVERE THUNDERSTORM WARN... http://t.co/1EOf1Wxnpj #Skywarn #OKwx


Target: 1
Text:
Severe Thunderstorm Warning including Russellville AR Clarksville AR Dardanelle AR until 10:15 PM CDT http://t.co/n844h1ASPj


Target: 1
Text:
RT @LivingSafely: NWS posts Severe #Thunderstorm Warnings for parts of #AR #NC #OK. Seek strong shelter if at risk: http://t.co/kEa5l3b1AE


Target: 1
Text:
Severe Thunderstorm Warning for Oklahoma County until 10:30pm. http://t.co/RsdkWZRc8g #okwx




### Spliting Data

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42)

In [16]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [17]:
temp = pd.DataFrame(data=train_sentences[:10], columns=["text"])
temp["target"] = train_labels[:10]
temp

,text,target
0,@mogacola @zamtriossu i screamed after hitting...,0
1,Imagine getting flattened by Kurt Zouma,0
2,@Gurmeetramrahim #MSGDoing111WelfareWorks Gree...,1
3,@shakjn @C7 @Magnums im shaking in fear he's g...,0
4,Somehow find you and I collide http://t.co/Ee8...,0
5,@EvaHanderek @MarleyKnysh great times until th...,1
6,destroy the free fandom honestly,1
7,Weapons stolen from National Guard Armory in N...,0
8,@wfaaweather Pete when will the heat wave pass...,1
9,Patient-reported outcomes in long-term survivo...,1


### Text vectorization (tokenization)

In [18]:
from tensorflow.keras.layers import TextVectorization

In [19]:
avg_sentence_length = round(sum([len(sentence.split()) for sentence in train_sentences]) / len(train_sentences))
avg_sentence_length

15

In [20]:
# this method's every parameter is set by default, what we need. Just need to change below two params.
text_vectorizer = TextVectorization(max_tokens=10000,
                                    output_sequence_length=avg_sentence_length)

In [21]:
text_vectorizer.adapt(train_sentences)

In [22]:
import random

In [23]:
random_sentence = random.choice(train_sentences)
print(f"Before tokenization: {random_sentence}")
print(f"After tokenization: {text_vectorizer([random_sentence])}")

Before tokenization: Do you have a plan in case of a pool chemical emergency? Learn more here: http://t.co/UePPjwvLcb #watersafety @CDC
After tokenization: [[  68   12   24    3  241    4  823    6    3 1616  554   73  944   51
   140]]


In [24]:
print(f"5 most common words: {text_vectorizer.get_vocabulary()[:5]}")
print(f"5 least common words: {text_vectorizer.get_vocabulary()[-5:]}")

5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Embedding

In [25]:
from tensorflow.keras.layers import Embedding

In [26]:
embedding = Embedding(input_dim=len(text_vectorizer.get_vocabulary()),
                      output_dim=128,
                      input_length=avg_sentence_length)

In [27]:
random_sentence = random.choice(train_sentences)
print(f"Sentence before embedding: {random_sentence}")
embedding(text_vectorizer([random_sentence]))

Sentence before embedding: Twitter is just here to destroy your childhood


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00225443,  0.03031803, -0.01507009, ..., -0.00689612,
         -0.04399556, -0.01313525],
        [ 0.0043386 ,  0.03237979, -0.00645089, ...,  0.02566281,
         -0.00200596, -0.03432401],
        [-0.00668384,  0.00394974, -0.03816146, ..., -0.03245485,
          0.03081815, -0.0103071 ],
        ...,
        [-0.02620229, -0.04226325, -0.04894365, ...,  0.0327565 ,
         -0.01877307,  0.00694592],
        [-0.02620229, -0.04226325, -0.04894365, ...,  0.0327565 ,
         -0.01877307,  0.00694592],
        [-0.02620229, -0.04226325, -0.04894365, ...,  0.0327565 ,
         -0.01877307,  0.00694592]]], dtype=float32)>

**Note:** If we see the shape of the embedded sentence, we will notice it is (1, 15, 128)<br>
which means from every sentence we will take 15 words and each word will be represented with 128 shape matrix.

### `model_0`: Baseline

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [29]:
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("multinomial", MultinomialNB())
])

In [30]:
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('multinomial', MultinomialNB())])

### Evaluation function

In [31]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [32]:
def calculate_results(y_true, y_pred):
    """
    returns a dictionary of accuracy_score, precission, recall and f1_score
    """
    precission, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    acc_score = accuracy_score(y_true, y_pred) * 100
    
    results = {
        "accuracy_score": acc_score,
        "precission": precission,
        "recall": recall,
        "f1_score": f1_score
    }
    return results

In [33]:
baseline_results = calculate_results(val_labels, model_0.predict(val_sentences))
baseline_results

{'accuracy_score': 79.26509186351706,
 'precission': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

### `model_1`: Simple dense model

In [34]:
import tensorflow as tf
from tensorflow.keras import layers

In [35]:
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.GlobalAveragePooling1D()(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [36]:
model_1.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [37]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [38]:
SAVE_DIR = "model_logs"

In [39]:
history_1 = model_1.fit(train_sentences, 
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                               experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230623-090212
Epoch 1/5
215/215 [==============================] - 26s 95ms/step - loss: 0.6373 - accuracy: 0.6459 - val_loss: 0.5736 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4668 - accuracy: 0.8292 - val_loss: 0.4743 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3405 - accuracy: 0.8751 - val_loss: 0.4566 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2582 - accuracy: 0.9070 - val_loss: 0.4572 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1980 - accuracy: 0.9339 - val_loss: 0.4676 - val_accuracy: 0.7874


In [40]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.45158088],
       [0.86350775],
       [0.9935481 ],
       [0.06967001],
       [0.31303054],
       [0.95369685],
       [0.9481982 ],
       [0.98511684],
       [0.9280556 ],
       [0.1343514 ]], dtype=float32)

In [41]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [42]:
model_1_result = calculate_results(y_true=val_labels,
                                 y_pred=model_1_preds)
model_1_result

{'accuracy_score': 78.74015748031496,
 'precission': 0.7923216200196284,
 'recall': 0.7874015748031497,
 'f1_score': 0.7844114973523961}

### `model_2`: LSTM

In [43]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(units=64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [44]:
model_2.compile(loss="binary_crossentropy", 
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [45]:
history_2 = model_2.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                              "LSTM_model")])

Saving TensorBoard log files to: model_logs/LSTM_model/20230623-091319
Epoch 1/5
215/215 [==============================] - 18s 61ms/step - loss: 0.3666 - accuracy: 0.8457 - val_loss: 0.5103 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2397 - accuracy: 0.9070 - val_loss: 0.4866 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1735 - accuracy: 0.9381 - val_loss: 0.5061 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1262 - accuracy: 0.9539 - val_loss: 0.8022 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0934 - accuracy: 0.9641 - val_loss: 1.0396 - val_accuracy: 0.7730


In [46]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10] 

24/24 [==============================] - 1s 4ms/step


array([[0.0043011 ],
       [0.7985897 ],
       [0.99974376],
       [0.03200303],
       [0.00110073],
       [0.9998896 ],
       [0.96720874],
       [0.99989736],
       [0.9998876 ],
       [0.13564934]], dtype=float32)

In [48]:
model_2_pred = tf.squeeze(tf.round(model_1_pred_probs))
model_2_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [50]:
model_2_result = calculate_results(y_true=val_labels,
                                  y_pred=model_2_pred)
model_2_result

{'accuracy_score': 78.74015748031496,
 'precission': 0.7923216200196284,
 'recall': 0.7874015748031497,
 'f1_score': 0.7844114973523961}

In [51]:
baseline_results

{'accuracy_score': 79.26509186351706,
 'precission': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

### `model_3': GRU

In [58]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
x = layers.GRU(units=64, return_sequences=True)(x)
# print(x.shape)
x = layers.GRU(units=64)(x)
# print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="GRU_model")

In [59]:
model_3.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [61]:
history_3 = model_3.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                             "GRU_model")])

Saving TensorBoard log files to: model_logs/GRU_model/20230623-093352
Epoch 1/5
215/215 [==============================] - 17s 58ms/step - loss: 0.1872 - accuracy: 0.9267 - val_loss: 0.6856 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.1038 - accuracy: 0.9613 - val_loss: 0.9439 - val_accuracy: 0.7612
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0648 - accuracy: 0.9721 - val_loss: 0.6939 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0578 - accuracy: 0.9730 - val_loss: 1.0219 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0532 - accuracy: 0.9750 - val_loss: 1.1570 - val_accuracy: 0.7703


In [65]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:5]

24/24 [==============================] - 0s 3ms/step


array([[1.6434382e-03],
       [6.1189657e-01],
       [9.9967682e-01],
       [5.8673970e-02],
       [2.5903745e-04]], dtype=float32)

In [67]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [68]:
model_3_result = calculate_results(y_true=val_labels,
                                  y_pred=model_3_pred)
model_3_result

{'accuracy_score': 77.03412073490814,
 'precission': 0.7755630249535594,
 'recall': 0.7703412073490814,
 'f1_score': 0.7667059443150692}